In [2]:
import json
import os
import copy
import astroddpm

In [5]:
CONFIG_FOLDER = os.path.join(os.path.dirname(astroddpm.__file__),'config')
CONFIG_FOLDER

'/mnt/home/dheurtel/astroddpm/astroddpm/config'

In [6]:
ALL_MODELS = os.path.join(CONFIG_FOLDER,'MODELS.json')
with open(ALL_MODELS) as f:
    ALL_MODELS_DICT = json.load(f)
print(ALL_MODELS_DICT.keys())

dict_keys(['DefaultDDPM', 'DiscreteSBM_VPSDE_MHD_BPROJ_N_1000_bottleneck_32', 'DiscreteSBM_VPSDE_MHD_BPROJ_N_1000_bottleneck_16', 'DiscreteSBM_VPSDE_MHD_BPROJ_N_1000_bottleneck_8', 'DiscreteSBM_VPSDE_MHD_BPROJ_N_4000_bottleneck_32', 'DiscreteSBM_VPSDE_MHD_BPROJ_N_4000_bottleneck_16', 'DiscreteSBM_VPSDE_MHD_BPROJ_N_4000_bottleneck_8', 'DiscreteSBM_VPSDE_MHD_BPROJ_N_8000_bottleneck_32', 'DiscreteSBM_VPSDE_MHD_BPROJ_N_8000_bottleneck_16', 'DiscreteSBM_VPSDE_MHD_BPROJ_N_8000_bottleneck_8', 'DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32', 'DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16', 'DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_8', 'DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_32', 'DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_16', 'DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_8', 'DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_8000_bottleneck_32', 'DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_8000_bottleneck_16', 'DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_8000_bottleneck_8']

In [7]:
config_init = ALL_MODELS_DICT['DefaultDDPM']

config_init

{'model_id': 'DefaultDDPM',
 'diffusion_model': {'sde': {'type': 'DiscreteVPSDE',
   'beta_min': 0.1,
   'beta_max': 20.0,
   'ddpm_math': True,
   'N': 1000},
  'network': {'in_c': 1,
   'out_c': 1,
   'first_c': 10,
   'sizes': [256, 128, 64, 32],
   'num_blocks': 1,
   'n_steps': 1000,
   'time_emb_dim': 100,
   'dropout': 0,
   'attention': [],
   'normalisation': 'default',
   'padding_mode': 'circular',
   'eps_norm': 1e-05,
   'skiprescale': True,
   'type': 'ResUNet'},
  'type': 'DiscreteSBM'},
 'dataloaders': {'train_batch_size': 64,
  'test_batch_size': 128,
  'num_workers': 8,
  'seed': 33,
  'split': 0.1,
  'dataset': {'dir': '/mnt/home/dheurtel/ceph/01_data/CATS_MHD_BPROJ_DENSITY/',
   'type': 'npy files',
   'transforms': ['Random_Rotate90'],
   'name': 'CATS_MHD_BPROJ_DENSITY'}},
 'optimizer': {'type': 'Adam', 'lr': 0.001, 'weight_decay': 0},
 'scheduler': {'type': 'invsqrt',
  'warmup': 100,
  'maxlr': 0.002,
  'last_step': -1},
 'ckpt_epoch': 200,
 'ckpt_dir': '/mnt/ho

## EXP1

In [20]:
sizes_list = [[256, 128, 64, 32], [256, 128, 64, 32, 16,], [256, 128, 64, 32, 16, 8,]]
N_list = [1000, 4000, 8000]
sde_list = ['DiscreteVPSDE', 'DiscreteSigmaVPSDE']

In [21]:
power_spectrum_name = 'cmb_256_8arcmippixel'

In [22]:
all_configs = {}
for sde in sde_list:
    for n in N_list:
        for sizes in sizes_list:
            config = copy.deepcopy(config_init)
            config['diffusion_model']['sde']['type'] = sde
            config['diffusion_model']['sde']['N'] = n
            config['diffusion_model']['network']['n_steps'] = n
            if sde == 'DiscreteSigmaVPSDE':
                config['diffusion_model']['sde']['power_spectrum_name'] = power_spectrum_name
            config['diffusion_model']['network']['sizes'] = sizes
            if sde == 'DiscreteVPSDE':
                config['model_id'] = 'DiscreteSBM_VPSDE_MHD_BPROJ_N_{}_bottleneck_{}'.format(n,sizes[-1])
            else:
                config['model_id'] = 'DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_{}_bottleneck_{}'.format(n,sizes[-1])
            all_configs[config['model_id']] = config
            

In [23]:
with open(os.path.join(CONFIG_FOLDER,'exp1','config.json'), 'w') as f:
    json.dump(all_configs, f, indent=4)

In [24]:
for key in all_configs.keys():
    print(key, all_configs[key]['diffusion_model']['sde']['N'], all_configs[key]['diffusion_model']['sde']['type'], all_configs[key]['diffusion_model']['network']['n_steps'], all_configs[key]['diffusion_model']['network']['sizes'][-1])

DiscreteSBM_VPSDE_MHD_BPROJ_N_1000_bottleneck_32 1000 DiscreteVPSDE 1000 32
DiscreteSBM_VPSDE_MHD_BPROJ_N_1000_bottleneck_16 1000 DiscreteVPSDE 1000 16
DiscreteSBM_VPSDE_MHD_BPROJ_N_1000_bottleneck_8 1000 DiscreteVPSDE 1000 8
DiscreteSBM_VPSDE_MHD_BPROJ_N_4000_bottleneck_32 4000 DiscreteVPSDE 4000 32
DiscreteSBM_VPSDE_MHD_BPROJ_N_4000_bottleneck_16 4000 DiscreteVPSDE 4000 16
DiscreteSBM_VPSDE_MHD_BPROJ_N_4000_bottleneck_8 4000 DiscreteVPSDE 4000 8
DiscreteSBM_VPSDE_MHD_BPROJ_N_8000_bottleneck_32 8000 DiscreteVPSDE 8000 32
DiscreteSBM_VPSDE_MHD_BPROJ_N_8000_bottleneck_16 8000 DiscreteVPSDE 8000 16
DiscreteSBM_VPSDE_MHD_BPROJ_N_8000_bottleneck_8 8000 DiscreteVPSDE 8000 8
DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32 1000 DiscreteSigmaVPSDE 1000 32
DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16 1000 DiscreteSigmaVPSDE 1000 16
DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_8 1000 DiscreteSigmaVPSDE 1000 8
DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_32 4000 Discret

In [19]:
## Adding the content of config.json to MODELS.json because I forgot to do it before
with open(os.path.join(CONFIG_FOLDER,'exp1','config.json')) as f:
    all_configs = json.load(f)

with open(ALL_MODELS) as f:
    ALL_MODELS_DICT = json.load(f)

ALL_MODELS_DICT.update(all_configs)

with open(ALL_MODELS, 'w') as f:
    json.dump(ALL_MODELS_DICT, f, indent=4)

### Summary:

## EXP2

In [16]:
sizes_list = [[256, 128, 64, 32], [256, 128, 64, 32, 16,]]
N_list = [1000, 4000]
sde_list = ['DiscreteSigmaVPSDE']
first_c_list = [4, 6, 10]

In [17]:
power_spectrum_name = 'cmb_256_8arcmippixel'

In [18]:
all_configs = {}
for sde in sde_list:
    for n in N_list:
        for sizes in sizes_list:
            for first_c in first_c_list:
                config = copy.deepcopy(config_init)
                config['diffusion_model']['sde']['type'] = sde
                config['diffusion_model']['sde']['N'] = n
                config['diffusion_model']['network']['n_steps'] = n
                config['diffusion_model']['network']['first_c'] = first_c
                config['diffusion_model']['sde']['power_spectrum_name'] = power_spectrum_name
                config['diffusion_model']['network']['sizes'] = sizes
                config['epochs'] = 20000
                config['model_id'] = 'DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_{}_bottleneck_{}_firstc_{}'.format(n,sizes[-1], first_c)
                all_configs[config['model_id']] = config

In [21]:
with open(os.path.join(CONFIG_FOLDER,'exp2','config.json'), 'w') as f:
    json.dump(all_configs, f, indent=4)

In [22]:
for key in all_configs.keys():
    print(key)

DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32_firstc_4
DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32_firstc_6
DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32_firstc_10
DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16_firstc_4
DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16_firstc_6
DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16_firstc_10
DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_32_firstc_4
DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_32_firstc_6
DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_32_firstc_10
DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_16_firstc_4
DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_16_firstc_6
DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_16_firstc_10


In [ ]:
## Adding the content of config.json to MODELS.json because I forgot to do it before
with open(os.path.join(CONFIG_FOLDER,'exp1','config.json')) as f:
    all_configs = json.load(f)

with open(ALL_MODELS) as f:
    ALL_MODELS_DICT = json.load(f)

ALL_MODELS_DICT.update(all_configs)

with open(ALL_MODELS, 'w') as f:
    json.dump(ALL_MODELS_DICT, f, indent=4)